In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("Social_Network_Ads.csv")

In [3]:
data=pd.get_dummies(data,drop_first=True)

In [7]:
data.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [8]:
indep=data[['User ID', 'Age', 'EstimatedSalary','Gender_Male']]

In [9]:
dep=data[['Purchased']]

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [16]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
parm={'n_neighbors':[5],'weights':['uniform'],'algorithm':['auto','ball_tree','kd_tree','brute'],'metric':['minkowski']}
grid=GridSearchCV(KNeighborsClassifier(),parm,refit=True,verbose=3,n_jobs=-1,scoring="f1_weighted")
grid.fit(x_train,y_train)

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:    3.9s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    4.0s finished
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['minkowski'], 'n_neighbors': [5],
                         'weights': ['uniform']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1_weighted', verbose=3)

In [24]:
re=grid.cv_results_
print(re)

{'mean_fit_time': array([0.02000443, 0.00666833, 0.00500234, 0.00666801]), 'std_fit_time': array([4.08403640e-03, 2.35820368e-03, 2.35486545e-06, 2.35758233e-03]), 'mean_score_time': array([0.0316759 , 0.01000961, 0.01166598, 0.02185607]), 'std_score_time': array([0.01247558, 0.00409339, 0.00623764, 0.00623584]), 'param_algorithm': masked_array(data=['auto', 'ball_tree', 'kd_tree', 'brute'],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'param_metric': masked_array(data=['minkowski', 'minkowski', 'minkowski', 'minkowski'],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'param_n_neighbors': masked_array(data=[5, 5, 5, 5],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'param_weights': masked_array(data=['uniform', 'uniform', 'uniform', 'uniform'],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=

In [25]:
y_pred=grid.predict(x_test)

In [28]:
from sklearn.metrics import confusion_matrix
cfm=confusion_matrix(y_test,y_pred)
from sklearn.metrics import classification_report
clfr=classification_report(y_test,y_pred)
print(cfm,clfr)

[[79  0]
 [41  0]]               precision    recall  f1-score   support

           0       0.66      1.00      0.79        79
           1       0.00      0.00      0.00        41

    accuracy                           0.66       120
   macro avg       0.33      0.50      0.40       120
weighted avg       0.43      0.66      0.52       120



C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [32]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average="weighted")
print("F1 score and best parameter{}:",format(grid.best_params_),f1_macro)

F1 score and best parameter{}: {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 5, 'weights': 'uniform'} 0.5226968174204355


C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [33]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
print(roc)

0.5


In [36]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_metric,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.020004,0.004084,0.031676,0.012476,auto,minkowski,5,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.880675,0.859759,0.878246,0.872921,0.009335,1
1,0.006668,0.002358,0.010010,0.004093,ball_tree,minkowski,5,uniform,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.880675,0.859759,0.878246,0.872921,0.009335,1
2,0.005002,0.000002,0.011666,0.006238,kd_tree,minkowski,5,uniform,"{'algorithm': 'kd_tree', 'metric': 'minkowski'...",0.880675,0.859759,0.878246,0.872921,0.009335,1
3,0.006668,0.002358,0.021856,0.006236,brute,minkowski,5,uniform,"{'algorithm': 'brute', 'metric': 'minkowski', ...",0.880675,0.859759,0.878246,0.872921,0.009335,1
